## 스크랩한 파일에 대한 DB 처리 프로세스

1. DB 프로세스
    - CSV 파일 폴더 읽기
    - 에러 체크
        * 에러시 : 에러파일 해당 파일 해당 폴더 저장, 정기 모니터링시 장기 미처리 파일 오류 체크 및 처리
        * csv 파일 읽어 DB 처리[테이블에 데이터 입력]
            + 해당 파일을 헤더와 CSV 데이터로 분리하여 읽어오기
            + CSV 부분 CSV로 변환후 DB 처리
        * 처리 완료 파일 이동[보관 기준?]
        * 개발시 sqllite 로 개발 처리
        * 배치 JOB 을 감안하여 RDB 방식으로 처리

### 스키마
- 테이블 명 : airfare_scraped_data
- 필드
    * 키
    * 스크래핑 날짜   : yyyyddmmhh
    * 스크래핑 사이트 : 2자리 코드
    * 패턴            : 1 - 편도 기준, 2 - 왕복 기준 , . . .
    * 대상항공사 코드 : 2자리 코드 - 조회대상 항공사 코드
    * 플라이트        : 플라이트 코드
    * 출발지          : 3자리 코드
    * 도착지          : 3자리 코드
    * 출발일          : yyyymmdd
    * 출발시간        : hhmm
    * 도착시간        : hhmm
    * 항공료          : .2
    * 유류세          : .2
    * 공항세          : .2
    * 좌석            : 0~9 숫자

### DB 스키마에 맞게 데이터 조정
- scrap_data 폴더에서 리스트를 생성하여 데이터 베이스 입력 처리
- 파일을 읽어 데이터 내용을 DB 스키마에 맞게 수정
    * null 처리(가격 정보 0으로 처리)
    * 날짜 데이터 YYYYMMDD 형식, 시간 데이터 HHMM 형식

In [1]:
import sqlite3
import csv, io
from datetime import datetime
from common.env_variable import SCRAP_DATA_DIR, DB_OK_DIR
from common.batch_util import get_crawled_file_list,read_crawled_file,move_scraped_file
from common.log_util import *
## 로그 초기화
logger_initialize('db_logger_setting.json')

In [2]:
def batch_db(scrap_dir,db_ok_dir,db):
    start_time = datetime.today()
    log_msgs = ['start batch job','doing db job with scraped data file']
    log(log_msgs,level=logging.INFO)
    ## 파일 리스트 생성
    db_cnt = 0
    files = get_crawled_file_list(scrap_dir)
    ## 각 파일 처리
    for file in files:
        log(file,level=logging.DEBUG)
        csv_file = scrap_dir+'/'+file
        head, raw_data = read_crawled_file(csv_file,csv=True)
        ## 스키마 체크 - 차후 오류 로그 체크하여 해당 내용 확인후 반영
        ## DB 처리
        log('execute insert query',level=logging.DEBUG)
        i_cnt = scraped_csv_to_db(head,raw_data,db)
        log(['insert result',i_cnt],level=logging.DEBUG)
        db_cnt += i_cnt
        ## 정상 처리 파일 처리
        if i_cnt > 0:
            move_scraped_file(csv_file,db_ok_dir)
    end_time = datetime.today()
    log_msgs = ['end batch job','doing db job with scraped data file',
                'elapsed -{}'.format(end_time-start_time),'Total :{} rows inserted.'.format(db_cnt)]
    log(log_msgs,level=logging.INFO)
    
def scraped_csv_to_db(head,raw_data,db):
    log('$$ check csv to db process.',level=logging.INFO)
    ## CSV 파일 읽기
    csv_data = csv.reader(io.StringIO(raw_data))
    ## DB 처리 리스트 생성
    target_list = []
    for d in csv_data:
        ## scrap_date, scrap_site, patten(1 편도)
        td_list = [head['crawl_date'],head['site_code'],'1']
        ## airline,flt,dpt,arr,dpt_time,arr_time,fare,tax1,tax2,seat
        td_list.extend(d)
        target_list.append(td_list)
    ## DB 처리
    cnt = 0
    conn = sqlite3.connect(db)
    try:
        cur = conn.cursor()
        sql = "insert into airfare_scraped_data"+\
        "(scrap_date,scrap_site,patten,airline,flt,dpt_date,dpt,arr,dpt_time,arr_time,fare,tax1,tax2,seat)"+\
        "values(?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
        cur.executemany(sql,target_list)
        conn.commit()
        cnt = len(target_list)
    except sqlite3.Error as e:
        if conn:
            conn.rollback()
        log(e,level=logging.ERROR)
        cnt = 0
    finally:
        if conn:
            conn.close()
    log('$$ check db process result : {}'.format(cnt),level=logging.INFO)
    return cnt

In [3]:
scrap_dir = SCRAP_DATA_DIR#'scrap_test'
db_ok_dir = DB_OK_DIR#'db_ok'
batch_db(scrap_dir,db_ok_dir,'airfare_scraped_data.db')

2017-07-07 08:50:48,767 root     INFO     start batch job :: doing db job with scraped data file
2017-07-07 08:50:48,809 root     DEBUG    scrap_2017070708_IP_7C_CJJ_CJU_20170708.csv
2017-07-07 08:50:48,822 root     DEBUG    execute insert query
2017-07-07 08:50:48,831 root     INFO     $$ check csv to db process.
2017-07-07 08:50:48,870 root     INFO     $$ check db process result : 6
2017-07-07 08:50:48,877 root     DEBUG    insert result :: 6
2017-07-07 08:50:48,885 root     DEBUG    scrap_2017070708_IP_7C_CJJ_CJU_20170709.csv
2017-07-07 08:50:48,898 root     DEBUG    execute insert query
2017-07-07 08:50:48,905 root     INFO     $$ check csv to db process.
2017-07-07 08:50:48,942 root     INFO     $$ check db process result : 6
2017-07-07 08:50:48,955 root     DEBUG    insert result :: 6
2017-07-07 08:50:48,975 root     DEBUG    scrap_2017070708_IP_7C_CJJ_CJU_20170710.csv
2017-07-07 08:50:48,986 root     DEBUG    execute insert query
2017-07-07 08:50:48,989 root     INFO     $$ che